In [1]:
# Database connection
import sqlalchemy
import json
import pandas as pd

def create_db_connection(args):
    """
    Creates a connection to the database specified in args
    :return: Returns the active connection
    """
    username = args['username']
    password = args['password']
    server = args['server']
    database = args['database']

    engine = sqlalchemy.create_engine(
        f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL Server&Trusted_Connection=yes')
    conn = engine.connect()
    return conn


def update_table(connection, table_name, data):
    """
    Discards old data and inserts the new into the SQL Server
    """
    data.to_sql(table_name, connection, if_exists='replace', index=False)

    return data

In [2]:
# Create connection from db file info
db_file = 'db.json'
with open(db_file) as db:
    connection = create_db_connection(json.load(db))

matches = pd.read_csv('datasets/matches.csv')
matches = update_table(connection, 'Match', matches)
print(matches.shape)

players = pd.read_csv('datasets/all_players.csv')
players = update_table(connection, 'PlayerStats', players)
print(players.shape)

goalkeepers = pd.read_csv('datasets/gk_players.csv')
goalkeepers = update_table(connection, 'GoalkeepingStats', goalkeepers)
print(goalkeepers.shape)

(317, 17)
(1332, 78)
(106, 34)
